In [1]:
%load_ext autoreload
%autoreload 2

# ExpectationValue

In [2]:
import numpy as np

In [3]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.opflow import PauliSumOp

observable = PauliSumOp.from_list([("XX", 1), ("YY", 2), ("ZZ", 3)])
print("observable\n", observable)

ansatz = RealAmplitudes(num_qubits=2, reps=2)
print("ansatz\n", ansatz)

observable
 1.0 * XX
+ 2.0 * YY
+ 3.0 * ZZ
ansatz
      ┌──────────┐     ┌──────────┐     ┌──────────┐
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├──■──┤ Ry(θ[4]) ├
     ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├
     └──────────┘└───┘└──────────┘└───┘└──────────┘


In [4]:
from qiskit.quantum_info import Statevector

expval = Statevector(ansatz.bind_parameters([0, 1, 1, 2, 3, 5])).expectation_value(
    observable.primitive
)
print(expval.real)

1.8420921273153716


/home/ima/github/qiskit/terra/qiskit/quantum_info/states/statevector.py:444: DeprecationWarning: The SparsePauliOp.table method is deprecated as of Qiskit Terra 0.19.0 and will be removed no sooner than 3 months after the releasedate. Use SparsePauliOp.paulis method instead.
  for z, x, coeff in zip(oper.table.Z, oper.table.X, oper.coeffs)


## ExpectationValue class

In [5]:
from qiskit.evaluators import PauliExpectationValue

### PauliExpectationValue

Evaluate the expectation value by sampling. This supports both AerSimulator and IBMQ backends.

In [6]:
from qiskit.providers.aer import AerSimulator

backend = AerSimulator()

In [7]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([0, 1, 1, 2, 3, 5], shos=1000)

no max_experiments for backend 'aer_simulator'. Set 1000000 as max_experiments.


ExpectationValueResult(value=1.90625, variance=11.133995056152344, confidence_interval=(1.746764707688813, 2.065735292311187))

In [8]:
# pre-binding

circuit = ansatz.bind_parameters([0, 1, 1, 2, 3, 5])
expval = PauliExpectationValue(circuit, observable, backend=backend)
expval.evaluate()

no max_experiments for backend 'aer_simulator'. Set 1000000 as max_experiments.


ExpectationValueResult(value=1.802734375, variance=11.04873275756836, confidence_interval=(1.6454290994023621, 1.9600396505976379))

In [9]:
# multi prameters

expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8]], shots=3000)

no max_experiments for backend 'aer_simulator'. Set 1000000 as max_experiments.


ExpectationValueArrayResult(values=array([1.934     , 0.09466667]), variances=array([11.08060044, 10.35124178]), confidence_intervals=array([[1.84133183e+00, 2.02666817e+00],
       [1.34590858e-03, 1.87987425e-01]]))

In [10]:
# can pass ndarray
expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate(np.array([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8]]), shots=8192)

no max_experiments for backend 'aer_simulator'. Set 1000000 as max_experiments.


ExpectationValueArrayResult(values=array([1.74829102, 0.16796875]), variances=array([11.21945018, 10.56518257]), confidence_intervals=array([[1.69178219, 1.80479984],
       [0.11107193, 0.22486557]]))

### Exact simulation by SaveExpectationValueVariance

In [11]:
from qiskit.evaluators import ExactExpectationValue

In [12]:
expval = ExactExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=1.8420921273153712, variance=6.432763522806666, confidence_interval=None)

In [13]:
expval = ExactExpectationValue(ansatz, observable, backend=backend)
expval.evaluate(np.array([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8]]))

ExpectationValueArrayResult(values=array([1.84209213, 0.15611897]), variances=array([6.43276352, 5.35681431]), confidence_intervals=array([None, None], dtype=object))

### Transpiled Circuits

In [14]:
from qiskit.test.mock import FakeBogota

backend = AerSimulator.from_backend(FakeBogota())

expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.set_transpile_options(initial_layout=[3, 2])
for circ in expval.transpiled_circuits:
    print(circ)
    print(circ.metadata)
    print()
print(expval.evaluate([0, 1, 1, 2, 3, 5]))

global phase: π
                                                                »
ancilla_0 -> 0 ─────────────────────────────────────────────────»
                                                                »
ancilla_1 -> 1 ─────────────────────────────────────────────────»
               ┌────┐┌──────────────┐┌────┐┌────────┐┌───┐┌────┐»
      q_1 -> 2 ┤ √X ├┤ Rz(θ[1] + π) ├┤ √X ├┤ Rz(3π) ├┤ X ├┤ √X ├»
               ├────┤├──────────────┤├────┤├────────┤└─┬─┘├────┤»
      q_0 -> 3 ┤ √X ├┤ Rz(θ[0] + π) ├┤ √X ├┤ Rz(3π) ├──■──┤ √X ├»
               └────┘└──────────────┘└────┘└────────┘     └────┘»
ancilla_2 -> 4 ─────────────────────────────────────────────────»
                                                                »
         c4: 2/═════════════════════════════════════════════════»
                                                                »
«                                                                          »
«ancilla_0 -> 0 ─────────────────────────────────

### Transpile options and Run options

In [15]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
# setter
expval.set_transpile_options(optimization_level=2)
expval.set_run_options(shots=100_000)
expval.evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=1.1815936565709086, variance=12.773175261790158, confidence_interval=(1.1636724525412137, 1.1995148606006036))

In [16]:
# Method chain
expval.set_run_options(shots=8192).evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=1.160400390625, variance=12.807823598384857, confidence_interval=(1.0976745168892448, 1.2231262643607552))

In [17]:
# evaluate's option
expval.evaluate([0, 1, 1, 2, 3, 5], shots=300)

ExpectationValueResult(value=1.3533333333333337, variance=12.614355555555555, confidence_interval=(1.0287160293412434, 1.677950637325424))

### Composite Evaluator

In [18]:
from qiskit.evaluators import JointEvaluator

In [19]:
len(expval.transpiled_circuits)

3

In [20]:
joint_evaluator = JointEvaluator([expval, expval, expval])  # can be different evaluator

In [21]:
len(joint_evaluator.transpiled_circuits)  # 3 × 3

9

In [22]:
joint_evaluator.evaluate([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8], [1, 2, 3, 5, 8, 13]])

CompositeResult(items=[ExpectationValueResult(value=1.2685546875, variance=12.71976613998413, confidence_interval=(1.20607736441788, 1.33103201058212)), ExpectationValueResult(value=1.2685546875, variance=12.71976613998413, confidence_interval=(1.20607736441788, 1.33103201058212)), ExpectationValueResult(value=1.2685546875, variance=12.71976613998413, confidence_interval=(1.20607736441788, 1.33103201058212))])

### Readout error mitigation

In [23]:
from qiskit.evaluators.backends import ReadoutErrorMitigation

backend = AerSimulator.from_backend(FakeBogota())
mit_tensored = ReadoutErrorMitigation(
    backend, mitigation="tensored", refresh=600, shots=2000, mit_pattern=[[0], [1]]
)
mit_mthree = ReadoutErrorMitigation(
    backend, mitigation="mthree", refresh=600, shots=2000, qubits=[0, 1]
)
expval_raw = PauliExpectationValue(ansatz, observable, backend=backend)
expval_tensored = PauliExpectationValue(ansatz, observable, backend=mit_tensored)
expval_mthree = PauliExpectationValue(ansatz, observable, backend=mit_mthree)
shots = 4000
print(
    f"w/o mitigation shots={shots}, result={expval_raw.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}"
)
print(
    f"w/ tensored mitigation shots={shots}, result={expval_tensored.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}"
)
print(
    f"w/ M3 mitigation shots={shots}, result={expval_mthree.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}"
)

w/o mitigation shots=4000, result=ExpectationValueResult(value=1.1660000000000001, variance=12.737613999999999, confidence_interval=(1.076577593355325, 1.2554224066446753))
w/ tensored mitigation shots=4000, result=ExpectationValueResult(value=1.3222693007615156, variance=12.271746241368245, confidence_interval=(1.235185111716235, 1.4093534898067963))
w/ M3 mitigation shots=4000, result=ExpectationValueResult(value=1.2995246051637028, variance=12.513545629371137, confidence_interval=(1.2111722492928154, 1.3878769610345902))


### Gradient of expectation value

In [24]:
from qiskit.evaluators.expectation_value.expectation_value_gradient import (
    FiniteDiffGradient,
    ParameterShiftGradient,
)

In [25]:
exact_expval = ExactExpectationValue(ansatz, observable, backend=AerSimulator())
exact_findiff = FiniteDiffGradient(exact_expval, 1e-8)
print(f"fin diff of exact {exact_findiff.evaluate([0, 1, 1, 2, 3, 5]).values}")

shots = 2000
findiff = FiniteDiffGradient(expval_raw, 1e-1)
paramshift = ParameterShiftGradient(expval_raw)
print(f"fin diff w/o mit {findiff.evaluate([0, 1, 1, 2, 3, 5], shots=shots).values}")
print(
    f"param shift w/o mit {paramshift.evaluate([0, 1, 1, 2, 3, 5], shots=shots).values}"
)

findiff = FiniteDiffGradient(expval_mthree, 1e-1)
paramshift = ParameterShiftGradient(expval_mthree)
print(f"fin diff w/  mit {findiff.evaluate([0, 1, 1, 2, 3, 5], shots=shots).values}")
print(
    f"param shift w/  mit {paramshift.evaluate([0, 1, 1, 2, 3, 5], shots=shots).values}"
)

fin diff of exact [0.13510328 1.87744376 1.03215425 1.87744378 1.48950681 2.15662828]
fin diff w/o mit [0.41 1.02 0.45 1.79 0.15 2.39]
param shift w/o mit [0.1165 1.278  0.771  1.2305 0.99   1.3815]
fin diff w/  mit [-0.52674771  0.23315659 -0.17908149  0.81220057 -0.72040078 -0.13687848]
param shift w/  mit [0.10313994 1.51538853 0.77419655 1.4110108  1.08052188 1.66335851]


### VQE by Scipy optimizer

In [26]:
from scipy.optimize import minimize

shot = 10000
backend = AerSimulator()
expval = PauliExpectationValue(ansatz, observable, backend=backend)
paramshift = ParameterShiftGradient(expval)
# this may take a long time...
result = minimize(
    lambda x: expval.evaluate(x, shots=shots).value,
    np.zeros(6),
    jac=lambda x: paramshift.evaluate(x, shots=shots).values,
)
print(result)

no max_experiments for backend 'aer_simulator'. Set 1000000 as max_experiments.


      fun: -6.0
 hess_inv: array([[ 0.64615801,  0.06965061,  0.09222648,  0.16811003, -0.40872812,
        -0.09849421],
       [ 0.06965061,  0.38046373, -0.11573058, -0.25182067,  0.04490767,
         0.31719274],
       [ 0.09222648, -0.11573058,  0.44997336,  0.35464395, -0.15096289,
        -0.09132804],
       [ 0.16811003, -0.25182067,  0.35464395,  0.52147463, -0.1706415 ,
        -0.2112584 ],
       [-0.40872812,  0.04490767, -0.15096289, -0.1706415 ,  0.48772413,
         0.31011013],
       [-0.09849421,  0.31719274, -0.09132804, -0.2112584 ,  0.31011013,
         0.67888704]])
      jac: array([ 0.171 ,  0.0245,  0.1355, -0.2005,  0.0425, -0.029 ])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 37
      nit: 10
     njev: 25
   status: 2
  success: False
        x: array([ 1.45412744, -0.25629983,  2.64921097,  1.07018779, -0.60697682,
       -2.47915899])
